In [1]:
from torch import nn
import torch
import torch.nn.functional as F

网络定义

In [3]:
#定义一个简单的前馈神经网络
net=nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
X=torch.randn(2,20)
Y=net(X)
Y

tensor([[ 0.0265, -0.0421, -0.0455, -0.0866, -0.3614,  0.0542, -0.1316,  0.1510,
          0.0149,  0.3155],
        [ 0.2061,  0.1507, -0.0176, -0.0202, -0.3134,  0.0862, -0.0566, -0.0162,
          0.0289,  0.0046]], grad_fn=<AddmmBackward0>)

In [4]:
#读取某一层的参数
print(net[2].state_dict())
print(net[2].state_dict().keys())


OrderedDict([('weight', tensor([[-0.0241,  0.0497, -0.0591,  ..., -0.0311, -0.0587,  0.0420],
        [ 0.0525, -0.0274,  0.0605,  ...,  0.0304, -0.0228, -0.0089],
        [-0.0268,  0.0601,  0.0196,  ...,  0.0084, -0.0064,  0.0437],
        ...,
        [ 0.0578,  0.0126, -0.0383,  ...,  0.0331,  0.0485, -0.0322],
        [-0.0409,  0.0278,  0.0177,  ..., -0.0192, -0.0476,  0.0143],
        [-0.0564,  0.0595,  0.0237,  ..., -0.0256, -0.0297,  0.0408]])), ('bias', tensor([ 0.0546, -0.0274, -0.0587,  0.0582,  0.0236, -0.0069,  0.0080,  0.0548,
         0.0131,  0.0469]))])
odict_keys(['weight', 'bias'])


In [8]:
print([(name,param.shape) for name,param in net.named_parameters()])

[('0.weight', torch.Size([256, 20])), ('0.bias', torch.Size([256])), ('2.weight', torch.Size([10, 256])), ('2.bias', torch.Size([10]))]


参数初始化

In [13]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)
        nn.init.constant_(m.bias,42)
#apply就是接收一个函数，对里面的每个元素都用这个函数执行一遍。不一定要是初始化
# net.apply(init_normal) #对net的每一层都使用init_normal函数

#单独对各个层进行初始化
net[0].apply(init_normal) #对net的第0层都使用init_normal函数
#net[1]是ReLU层，没有权重参数
net[2].apply(init_42) #对net的地2层使用init_42函数


print(net[0].weight.data[0],'\n',net[0].bias.data[0])
print(net[2].weight.data[2],'\n',net[2].bias.data[2])


tensor([-0.0072,  0.0064, -0.0080,  0.0044,  0.0109,  0.0198,  0.0056, -0.0069,
        -0.0137,  0.0190,  0.0097, -0.0106,  0.0074, -0.0110, -0.0035,  0.0045,
         0.0011,  0.0088,  0.0016,  0.0252]) 
 tensor(0.)
tensor(0.0634)
tensor([42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
        42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42

#### 参数绑定

对于某些层的参数,通过参数绑定使两个不同的层具备完全相同的参数,减少参数量的同时也能保证性能

In [19]:
#定义共享层
shared=nn.Linear(8,8)
#定义网络
net=nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8,4),
    nn.ReLU(),
)
X=torch.randn(2,4)
net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])
print(net[2].weight.data[0,0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0])
print(net[2].weight.data[0,0])
#修改了也相同,说明这两层的参数是一致的

tensor([True, True, True, True, True, True, True, True])
tensor(0.0659)
tensor([True, True, True, True, True, True, True, True])
tensor(100.)
